In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
import plotly.express as px

In [6]:
data = pd.read_csv("E:/Machine Learning/Random Forest/Travel_Classification.csv")

In [7]:
data.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [8]:
data.isna().sum()

CustomerID                    0
ProdTaken                     0
Age                         226
TypeofContact                25
CityTier                      0
DurationOfPitch             251
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            45
ProductPitched                0
PreferredPropertyStar        26
MaritalStatus                 0
NumberOfTrips               140
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     66
Designation                   0
MonthlyIncome               233
dtype: int64

In [9]:
data['Gender'].value_counts()

Gender
Male       2916
Female     1817
Fe Male     155
Name: count, dtype: int64

In [10]:
data['MaritalStatus'].value_counts()

MaritalStatus
Married      2340
Divorced      950
Single        916
Unmarried     682
Name: count, dtype: int64

In [11]:
data['Gender'] = data['Gender'].str.replace("Fe Male", "Female")

In [12]:
data['MaritalStatus'] = data['Gender'].str.replace("Single", "Unmarried")

In [13]:
data['Gender'].value_counts()

Gender
Male      2916
Female    1972
Name: count, dtype: int64

In [14]:
data['MaritalStatus'].value_counts()

MaritalStatus
Male      2916
Female    1972
Name: count, dtype: int64

In [15]:
data['TypeofContact'].value_counts()

TypeofContact
Self Enquiry       3444
Company Invited    1419
Name: count, dtype: int64

In [16]:
features_with_na = [features for features in data.columns if data[features].isnull().sum()>1]
for feature in features_with_na:
    print(feature, np.round(data[feature].isnull().mean()*100,5), "% missing values")

Age 4.62357 % missing values
TypeofContact 0.51146 % missing values
DurationOfPitch 5.13502 % missing values
NumberOfFollowups 0.92062 % missing values
PreferredPropertyStar 0.53191 % missing values
NumberOfTrips 2.86416 % missing values
NumberOfChildrenVisiting 1.35025 % missing values
MonthlyIncome 4.76678 % missing values


In [17]:
data.columns

Index(['CustomerID', 'ProdTaken', 'Age', 'TypeofContact', 'CityTier',
       'DurationOfPitch', 'Occupation', 'Gender', 'NumberOfPersonVisiting',
       'NumberOfFollowups', 'ProductPitched', 'PreferredPropertyStar',
       'MaritalStatus', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore',
       'OwnCar', 'NumberOfChildrenVisiting', 'Designation', 'MonthlyIncome'],
      dtype='object')

In [18]:
data['Age'].fillna(data.Age.median(),inplace=True)
data['TypeofContact'].fillna(data.TypeofContact.mode()[0],inplace=True)
data['DurationOfPitch'].fillna(data.DurationOfPitch.median(),inplace=True)
data['NumberOfFollowups'].fillna(data.NumberOfFollowups.mode()[0],inplace=True)
data['PreferredPropertyStar'].fillna(data.PreferredPropertyStar.mode()[0],inplace=True)
data['NumberOfTrips'].fillna(data['NumberOfTrips'].median(),inplace=True)
data['NumberOfChildrenVisiting'].fillna(data.NumberOfChildrenVisiting.mode()[0],inplace=True)
data['MonthlyIncome'].fillna(data.MonthlyIncome.median(),inplace=True)

In [19]:
data.isna().sum()

CustomerID                  0
ProdTaken                   0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch             0
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
MaritalStatus               0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
OwnCar                      0
NumberOfChildrenVisiting    0
Designation                 0
MonthlyIncome               0
dtype: int64

In [20]:
data.drop('CustomerID',inplace=True,axis=1)

In [21]:
data.columns

Index(['ProdTaken', 'Age', 'TypeofContact', 'CityTier', 'DurationOfPitch',
       'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups',
       'ProductPitched', 'PreferredPropertyStar', 'MaritalStatus',
       'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'OwnCar',
       'NumberOfChildrenVisiting', 'Designation', 'MonthlyIncome'],
      dtype='object')

In [22]:
data['TotalVisiting'] = data['NumberOfPersonVisiting']+data[ 'NumberOfChildrenVisiting']
data.drop([ 'NumberOfChildrenVisiting','NumberOfPersonVisiting'],axis=1,inplace=True)

In [23]:
Num_features = [features for features in data.columns if data[features].dtype !='O']
print(len(Num_features))
print(Num_features)

12
['ProdTaken', 'Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome', 'TotalVisiting']


In [24]:
Cat_features = [features for features in data.columns if data[features].dtype =='O']
print(len(Cat_features))
print(Cat_features)

6
['TypeofContact', 'Occupation', 'Gender', 'ProductPitched', 'MaritalStatus', 'Designation']


In [25]:
discrete_features = [features for features in Num_features if len(data[features].unique()) <=25]
print(len(discrete_features))
print(discrete_features)

9
['ProdTaken', 'CityTier', 'NumberOfFollowups', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'OwnCar', 'TotalVisiting']


In [26]:
continuous_features = [features for features in Num_features if len(data[features].unique()) >25]
print(len(continuous_features))
print(continuous_features)

3
['Age', 'DurationOfPitch', 'MonthlyIncome']


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X= data.drop(['ProdTaken'],axis=1)
Y= data['ProdTaken']

In [29]:
Y.value_counts()

ProdTaken
0    3968
1     920
Name: count, dtype: int64

In [30]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((3910, 17), (978, 17))

In [31]:
Cat_features = X.select_dtypes(include='object').columns
Num_features = X.select_dtypes(exclude='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer([("OneHotEncodder",oh_transformer,Cat_features),
                                  ("StandardScaler",numeric_transformer,Num_features)])

In [32]:
X_train = preprocessor.fit_transform(X_train)

In [33]:
X_test = preprocessor.transform(X_test)

In [35]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

In [36]:
models = {"Random Forest": RandomForestClassifier(), 
          "Decision Tree": DecisionTreeClassifier(),
          "Adaboost":AdaBoostClassifier()}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)

    Y_train_Pred = model.predict(X_train)
    Y_test_Pred = model.predict(X_test)

    #Training Set
    model_train_accuracy = accuracy_score(Y_train, Y_train_Pred)
    model_train_f1 = f1_score(Y_train, Y_train_Pred, average='weighted')
    model_train_precision = precision_score(Y_train, Y_train_Pred)
    model_train_recall = recall_score(Y_train, Y_train_Pred)
    model_train_rocauc_score = roc_auc_score(Y_train, Y_train_Pred)


    #Test Set
    model_test_accuracy = accuracy_score(Y_test, Y_test_Pred)
    model_test_f1 = f1_score(Y_test, Y_test_Pred, average='weighted')
    model_test_precision = precision_score(Y_test, Y_test_Pred)
    model_test_recall = recall_score(Y_test, Y_test_Pred)
    model_test_rocauc_score = roc_auc_score(Y_test, Y_test_Pred)

    print(list(models.keys())[i])


    print("Model Performance for Training Set")
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print("- F1 Score: {:.4f}".format(model_train_f1))
    print("- Precision: {:.4f}".format(model_train_precision))
    print("- Recall: {:.4f}".format(model_train_recall))
    print("- Roc Auc Score: {:.4f}".format(model_train_rocauc_score))



    print("--------------------")


    print("Model Performance for Test Set")
    print("- Accuracy: {:.4f}".format(model_test_accuracy))
    print("- F1 Score: {:.4f}".format(model_test_f1))
    print("- Precision: {:.4f}".format(model_test_precision))
    print("- Recall: {:.4f}".format(model_test_recall))
    print("- Roc Auc Score: {:.4f}".format(model_test_rocauc_score))


Random Forest
Model Performance for Training Set
- Accuracy: 1.0000
- F1 Score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 1.0000
--------------------
Model Performance for Test Set
- Accuracy: 0.9294
- F1 Score: 0.9243
- Precision: 0.9621
- Recall: 0.6649
- Roc Auc Score: 0.8293
Decision Tree
Model Performance for Training Set
- Accuracy: 1.0000
- F1 Score: 1.0000
- Precision: 1.0000
- Recall: 1.0000
- Roc Auc Score: 1.0000
--------------------
Model Performance for Test Set
- Accuracy: 0.9172
- F1 Score: 0.9164
- Precision: 0.8022
- Recall: 0.7644
- Roc Auc Score: 0.8593
Adaboost
Model Performance for Training Set
- Accuracy: 0.8499
- F1 Score: 0.8279
- Precision: 0.7017
- Recall: 0.3388
- Roc Auc Score: 0.6529
--------------------
Model Performance for Test Set
- Accuracy: 0.8364
- F1 Score: 0.8109
- Precision: 0.6782
- Recall: 0.3089
- Roc Auc Score: 0.6367


In [45]:
#HyperParameter Tuning

rf_params = {"max_depth":[5,8,15,"None,10"], "max_features":[5,7,"auto",8],
             "min_samples_split":[2,8,15,20],"n_estimators":[100,200,500,1000]}

adaboost_params = {"n_estimators":[50,60,70,80,90],
                   "algorithm":['SAMME','SAMME.R']}

In [46]:
randomcvmodels = [("Adaboost", AdaBoostClassifier(), adaboost_params)]

In [47]:
randomcvmodels

[('Adaboost',
  AdaBoostClassifier(),
  {'n_estimators': [50, 60, 70, 80, 90], 'algorithm': ['SAMME', 'SAMME.R']})]

In [48]:
from sklearn.model_selection import RandomizedSearchCV
model_param = {}

for name, model, params in randomcvmodels:
    random = RandomizedSearchCV(estimator=model,param_distributions=params, n_iter=100, cv=3, verbose=2, n_jobs=-1)
    random.fit(X_train, Y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---Best Params for {model_name}---")
    print(model_param[model_name])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
---Best Params for Adaboost---
{'n_estimators': 80, 'algorithm': 'SAMME'}


In [51]:
models = {"Adaboost": AdaBoostClassifier(n_estimators=80,algorithm='SAMME')}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, Y_train)

    Y_train_Pred = model.predict(X_train)
    Y_test_Pred = model.predict(X_test)

    #Training Set
    model_train_accuracy = accuracy_score(Y_train, Y_train_Pred)
    model_train_f1 = f1_score(Y_train, Y_train_Pred, average='weighted')
    model_train_precision = precision_score(Y_train, Y_train_Pred)
    model_train_recall = recall_score(Y_train, Y_train_Pred)
    model_train_rocauc_score = roc_auc_score(Y_train, Y_train_Pred)


    #Test Set
    model_test_accuracy = accuracy_score(Y_test, Y_test_Pred)
    model_test_f1 = f1_score(Y_test, Y_test_Pred, average='weighted')
    model_test_precision = precision_score(Y_test, Y_test_Pred)
    model_test_recall = recall_score(Y_test, Y_test_Pred)
    model_test_rocauc_score = roc_auc_score(Y_test, Y_test_Pred)

    print(list(models.keys())[i])


    print("Model Performance for Training Set")
    print("- Accuracy: {:.4f}".format(model_train_accuracy))
    print("- F1 Score: {:.4f}".format(model_train_f1))
    print("- Precision: {:.4f}".format(model_train_precision))
    print("- Recall: {:.4f}".format(model_train_recall))
    print("- Roc Auc Score: {:.4f}".format(model_train_rocauc_score))



    print("--------------------")


    print("Model Performance for Test Set")
    print("- Accuracy: {:.4f}".format(model_test_accuracy))
    print("- F1 Score: {:.4f}".format(model_test_f1))
    print("- Precision: {:.4f}".format(model_test_precision))
    print("- Recall: {:.4f}".format(model_test_recall))
    print("- Roc Auc Score: {:.4f}".format(model_test_rocauc_score))


Adaboost
Model Performance for Training Set
- Accuracy: 0.8468
- F1 Score: 0.8132
- Precision: 0.7754
- Recall: 0.2510
- Roc Auc Score: 0.6172
--------------------
Model Performance for Test Set
- Accuracy: 0.8364
- F1 Score: 0.7977
- Precision: 0.7818
- Recall: 0.2251
- Roc Auc Score: 0.6049
